<h1>Face Detection and Privacy</h1>
This program focuses on creating an algorithm for detecting human faces under varying conditions, extracting the faces, and applying blurring to hide the eyes

In [14]:
import cv2
import matplotlib.pyplot as plt

def load_image(image_path):
    return cv2.imread(image_path)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def detect_faces(gray_image):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    return face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

def process_image(image_path):
    image = load_image(image_path)
    gray_image = convert_to_grayscale(image)
    faces = detect_faces(gray_image)
    return image, faces

def draw_rectangles(image, faces, color=(0, 0, 255)):
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    return image

def save_image_with_boxes(image, output_path):
    cv2.imwrite(output_path, image)

image_paths = ['musk_full_body.jpg', 'musk_other_object.jpg', 'musk_with_other.jpg', 'musk_distant.jpg']
for image_path in image_paths:
    image, faces = process_image(image_path)
    image_with_boxes = draw_rectangles(image, faces)
    output_path = "boxed_" + image_path
    save_image_with_boxes(image_with_boxes, output_path)
    print(f"Image saved to {output_path}")



Image saved to boxed_musk_full_body.jpg
Image saved to boxed_musk_other_object.jpg
Image saved to boxed_musk_with_other.jpg
Image saved to boxed_musk_distant.jpg


Crop only the faces identified with the red box and save to local disk

In [15]:
import cv2
import os

def load_image(image_path):
    return cv2.imread(image_path)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def detect_faces(gray_image):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    return face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

def process_image(image_path):
    image = load_image(image_path)
    gray_image = convert_to_grayscale(image)
    faces = detect_faces(gray_image)
    return image, faces

def crop_and_save_faces(image, faces, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for i, (x, y, w, h) in enumerate(faces):
        cropped_face = image[y:y+h, x:x+w]
        output_path = os.path.join(output_directory, f"face_{i}.jpg")
        cv2.imwrite(output_path, cropped_face)
        print(f"Cropped face saved to {output_path}")

image_paths = ['musk_full_body.jpg', 'musk_other_object.jpg', 'musk_with_other.jpg', 'musk_distant.jpg']
for image_path in image_paths:
    image, faces = process_image(image_path)
    output_directory = os.path.splitext(image_path)[0] + "_faces"
    crop_and_save_faces(image, faces, output_directory)



Cropped face saved to musk_full_body_faces\face_0.jpg
Cropped face saved to musk_other_object_faces\face_0.jpg
Cropped face saved to musk_with_other_faces\face_0.jpg
Cropped face saved to musk_with_other_faces\face_1.jpg
Cropped face saved to musk_with_other_faces\face_2.jpg
Cropped face saved to musk_with_other_faces\face_3.jpg
Cropped face saved to musk_distant_faces\face_0.jpg


Now, detect the eyes, apply blurring, and save the images to disk.

In [17]:
import cv2
import os

def load_image(image_path):
    return cv2.imread(image_path)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def detect_faces(gray_image):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    return face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

def detect_eyes(gray_image):
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    return eye_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=3, minSize=(10, 10), flags=cv2.CASCADE_SCALE_IMAGE)

def process_image(image_path):
    image = load_image(image_path)
    gray_image = convert_to_grayscale(image)
    faces = detect_faces(gray_image)
    return image, faces

def crop_and_save_faces(image, faces, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for i, (x, y, w, h) in enumerate(faces):
        cropped_face = image[y:y+h, x:x+w]
        output_path = os.path.join(output_directory, f"face_{i}.jpg")
        cv2.imwrite(output_path, cropped_face)
        print(f"Cropped face saved to {output_path}")

def resize_image(image, new_width=200):
    aspect_ratio = float(image.shape[1]) / float(image.shape[0])
    new_height = int(new_width / aspect_ratio)
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def detect_eyes_and_blur_faces(image_path, output_path):
    face = load_image(image_path)
    resized_face = resize_image(face)
    gray_face = convert_to_grayscale(resized_face)
    eyes = detect_eyes(gray_face)

    for (ex, ey, ew, eh) in eyes:
        face_roi = resized_face[ey:ey+eh, ex:ex+ew]
        blurred_roi = cv2.GaussianBlur(face_roi, (15, 15), 30)
        resized_face[ey:ey+eh, ex:ex+ew] = blurred_roi

    cv2.imwrite(output_path, resized_face)
    print(f"Face with blurred eyes saved to {output_path}")

image_paths = ['musk_full_body.jpg', 'musk_other_object.jpg', 'musk_with_other.jpg', 'musk_distant.jpg']
for image_path in image_paths:
    image, faces = process_image(image_path)
    output_directory = os.path.splitext(image_path)[0] + "_faces"
    crop_and_save_faces(image, faces, output_directory)

    for i in range(len(faces)):
        face_image_path = os.path.join(output_directory, f"face_{i}.jpg")
        blurred_output_path = os.path.join(output_directory, f"blurred_face_{i}.jpg")
        detect_eyes_and_blur_faces(face_image_path, blurred_output_path)



Cropped face saved to musk_full_body_faces\face_0.jpg
Face with blurred eyes saved to musk_full_body_faces\blurred_face_0.jpg
Cropped face saved to musk_other_object_faces\face_0.jpg
Face with blurred eyes saved to musk_other_object_faces\blurred_face_0.jpg
Cropped face saved to musk_with_other_faces\face_0.jpg
Cropped face saved to musk_with_other_faces\face_1.jpg
Cropped face saved to musk_with_other_faces\face_2.jpg
Cropped face saved to musk_with_other_faces\face_3.jpg
Face with blurred eyes saved to musk_with_other_faces\blurred_face_0.jpg
Face with blurred eyes saved to musk_with_other_faces\blurred_face_1.jpg
Face with blurred eyes saved to musk_with_other_faces\blurred_face_2.jpg
Face with blurred eyes saved to musk_with_other_faces\blurred_face_3.jpg
Cropped face saved to musk_distant_faces\face_0.jpg
Face with blurred eyes saved to musk_distant_faces\blurred_face_0.jpg
